In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import plotly.express as px
import numpy as np
import tqdm as tqdm
from torch.utils.data import DataLoader
import torch.amp as amp

if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev) 

In [6]:
INPUT_SIZE_1 = 3
OUTPUT_SIZE_1 = 1

# in:
#   weekday
#   today
#   product label
# out:
#   days in storage

labels = {
    0: 5030,
    1: 5409
}

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        INTERNAL = 1
        self.l1 = nn.Linear(INPUT_SIZE_1, INTERNAL)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(INTERNAL, OUTPUT_SIZE_1)
    
    def forward(self, x):
        return self.l2(self.relu(self.l1(x)))

In [7]:
class Data1(torch.utils.data.Dataset):
    def __init__(self, filepath):
        data = pd.read_csv(filepath)
        out = pd.DataFrame()
        out['product_id'] = data['product_id']
        out['production_date'] = data['PRODUCTION DATE'].map(lambda x: pd.to_datetime(x).value // 10**12)
        m = out['production_date'].min()
        out['days_in_storage'] = data['days_in_storage']
        out['weekday'] = data['weekday']
        self.data = out
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        x = row.drop(labels=['days_in_storage']).values
        y = row.drop(labels=['production_date', 'weekday', 'product_id']).values
        return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=torch.float)

data1 = Data1('./time.csv')

train_loader = DataLoader(
    data1,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

C:\Users\pix\AppData\Local\Temp\ipykernel_6732\826075446.py:6: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  out['production_date'] = data['PRODUCTION DATE'].map(lambda x: pd.to_datetime(x).value // 10**12)


In [8]:
model1 = Net1().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
scaler = amp.GradScaler('cuda')
epochs = 10

costval = []

for i in tqdm.tqdm(range(epochs)):
    for j, (x_train, y_train) in enumerate(data1):
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        with amp.autocast(device_type='cuda'):
            y_pred = model1(x_train)
            cost = criterion(y_pred, y_train)
        
        optimizer.zero_grad(set_to_none=True)
        scaler.scale(cost).backward()
        scaler.step(optimizer)
        scaler.update()
    if i % 1 == 0:
        torch.save(model1, "model1.pt")

torch.save(model1, "model1.pt")

100%|██████████| 10/10 [04:09<00:00, 24.90s/it]
